In [12]:
%%bash
pip install -U requests

  Found existing installation: requests 2.2.1
    Uninstalling requests-2.2.1:
      Successfully uninstalled requests-2.2.1


/usr/local/lib/python2.7/dist-packages/pip/_vendor/requests/packages/urllib3/util/ssl_.py:318: SNIMissingWarning: An HTTPS request has been made, but the SNI (Subject Name Indication) extension to TLS is not available on this platform. This may cause the server to present an incorrect TLS certificate, which can cause validation failures. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#snimissingwarning.
  SNIMissingWarning
/usr/local/lib/python2.7/dist-packages/pip/_vendor/requests/packages/urllib3/util/ssl_.py:122: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
    DEPRECATION: Un

In [1]:
# opengrid imports
from opengrid.library import misc, houseprint, caching
from opengrid.library.analysis import DailyAgg
from opengrid import config
from opengrid.library.slack import Slack
from opengrid.library import alerts
c=config.Config()

# other imports
import pandas as pd
import json
import charts
import numpy as np
import os
import datetime as dt
import pytz
BXL = pytz.timezone('Europe/Brussels')


# configuration for the plots
DEV = c.get('env', 'type') == 'dev' # DEV is True if we are in development environment, False if on the droplet
print("Environment configured for development: {}".format(DEV))
if not DEV:
    # production environment: don't try to display plots
    import matplotlib
    matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.dates import MinuteLocator, HourLocator, DateFormatter, AutoDateLocator, num2date

if DEV:
    if c.get('env', 'plots') == 'inline':
        %matplotlib inline
    else:
        %matplotlib qt
else:
    pass # don't try to render plots
plt.rcParams['figure.figsize'] = 12,8

Server running in the folder /usr/local/opengrid/notebooks/Analysis at 127.0.0.1:33461
Environment configured for development: True


In [2]:
hp = houseprint.Houseprint()
sensors = hp.get_sensors(sensortype='electricity') # sensor objects

# Remove some sensors
exclude = [
            '565de0a7dc64d8370aa321491217b85f' # 3E
          ]
solar = [x.key for x in hp.search_sensors(type='electricity', system='solar')]
exclude += solar

for s in sensors:
    if s.key in exclude:
        sensors.remove(s)

hp.init_tmpo()

Opening connection to Houseprint sheet
Opening spreadsheets
Parsing spreadsheets
24 Sites created
24 Devices created
75 sensors created
Houseprint parsing complete
Using tmpo database from /root/.tmpo/tmpo.sqlite3


In [3]:
hp.sync_tmpos()

/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:318: SNIMissingWarning: An HTTPS request has been made, but the SNI (Subject Name Indication) extension to TLS is not available on this platform. This may cause the server to present an incorrect TLS certificate, which can cause validation failures. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/security.html#snimissingwarning.
  SNIMissingWarning
/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:122: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
/usr/local/lib/python2.7/dist-packages/reques

In [4]:
dt_now = pd.Timestamp('now', tz='UTC')
dt_start = dt_now - pd.Timedelta(hours=24)
df = hp.get_data(sensors=sensors, head=dt_start)

In [5]:
df_total = pd.DataFrame(df.sum()/60/1000)
df_total.columns=['sum']

Sort the daily consumptions

In [6]:
df_total.sort(columns=['sum'], axis=0, inplace=True)
df_total['percentile'] = np.array(range(len(df_total)))/float(len(df_total))

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [7]:
df_total

,sum,percentile
8c80f676768ab471a20966c8a996a795,0.997767,0.000000
e1dec16012edc11de998aeaa317274d7,1.570264,0.038462
2dd0f6942224767b18a411415f67b423,2.112225,0.076923
cdd4808a8d0a228ba16fba7c66dda81a,2.616465,0.115385
dfb719cc8240de810862228a7d1f37f3,2.868575,0.153846
156e0abc64b00c5e98dbd3c17883b5f6,2.925725,0.192308
8520ae493bcc57574e09b6ff7b13be76,3.950395,0.230769
77aaf550776560cd11eb68499b500075,6.380917,0.269231
99d7f090c9b191e31067a5fa922da025,7.502599,0.307692
c1a78eacaa6a82d3257a278d3e99088a,8.059077,0.346154


# Setup percentile slack bot

In [8]:
slack_url = c.get('Slack', 'webhook')
username = 'PercentileBot'
channel = "junk" # we don't want to clutter up everything
emoji = ':warning:'
slack = Slack(url=slack_url, username=username, channel=channel, emoji=emoji)

# Setup absolute slack bot

In [9]:
slack_url = c.get('Slack', 'webhook')
username = 'AbsoluteBot'
channel = "junk" # we don't want to clutter up everything
emoji = ':warning:'
slack = Slack(url=slack_url, username=username, channel=channel, emoji=emoji)

# Create the alerts and send

In [10]:
alerts.create_alerts(df_total, hp, 'percentile_analysis', slack, column='percentile')
alerts.create_alerts(df_total, hp, 'absolute_analysis', slack, column='sum')

/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:122: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:122: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:122: InsecurePlatformWarning

# Attention, run this only once, to create a default alerts.cfg

In [ ]:
threshhold = {
    "percentile_analysis":
         {"default":0.8,
          "3aa40023a1683bfec0f01cce8d45fc70":0.5
         },
 "absolute_analysis":
     {"default":25,
      "3aa40023a1683bfec0f01cce8d45fc70":10.5
     }
}

In [ ]:

json.dump(threshhold, open("alerts.cfg", 'w'))

In [ ]:
df_total

In [ ]:
df['ad322ec64956ed232cb94a72517004ab'].plot()

In [ ]:
fig = plt.gcf()

In [ ]:
fig.savefig()